# Lab Basis Data Recruitment
# Ricky Wijaya
# 18222043

# Preparation

In [1]:
# import library yang diperlukan
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException,TimeoutException,NoSuchElementException
import json
import datetime
import csv

# Fungsi 

## Mengambil data statistik pemain IBL pada Regular Season 23/24 

In [3]:
def scrape_player_stats():
    try:
        data_list = []
        # Menuju tim profile page
        PATH = "C:\Program Files (x86)\chromedriver.exe"
        service = Service(PATH)
        # Inisialisasi chrome driver
        driver = webdriver.Chrome(service=service)
        driver.get('https://iblindonesia.com/profile/team')
        # tunggu sampai page load sempurna
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//a[contains(text(), "Statistic")]')))
        # Cari semua "Statistic" 
        statistic_buttons = driver.find_elements(By.XPATH, '//a[contains(text(), "Statistic")]')
        for i in range(len(statistic_buttons)):
            try:
                statistic_buttons = driver.find_elements(By.XPATH, '//a[contains(text(), "Statistic")]')
                statistic_buttons[i].click()
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//tbody//tr')))
                # Extract headers
                headers = driver.find_elements(By.XPATH, '//thead//th/span')
                header_texts = [header.get_attribute('title') for header in headers]
                header_texts.insert(0, 'Player') 
                # Extract rows
                rows = driver.find_elements(By.XPATH, '//tbody//tr')
                for row in rows:
                    cells = row.find_elements(By.TAG_NAME, 'td')
                    data_dict = {}
                    for header, cell in zip(header_texts, cells):
                        data_dict[header] = cell.text
                    data_list.append(data_dict)
                # balik ke team profile page
                driver.back()
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//a[contains(text(), "Statistic")]')))
            except Exception as e:
                print(f"An error occurred while processing a player: {e}")
                driver.back()
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//a[contains(text(), "Statistic")]'))) 
        return data_list
    finally:
        driver.quit()

In [4]:
# scrape player 
data = scrape_player_stats()

## Mengambil data klasemen tim pada regular season IBL 23/24

In [5]:
def scrape_team_stats():
    team_stats = []
    # ke standings page
    PATH = "C:\Program Files (x86)\chromedriver.exe"
    service = Service(PATH)
    # Inisiasi chrome driver
    driver = webdriver.Chrome(service=service)
    
    driver.get("https://iblindonesia.com/games/standing")
    
    # cari the table and header
    table = driver.find_element(By.TAG_NAME, 'table')
    header_rows = table.find_elements(By.TAG_NAME, 'tr')
    
    # Extract header dari row pertama
    headers_team = header_rows[0].find_elements(By.TAG_NAME, 'th')
    header_names_team = [header.text for header in headers_team]
    rows = driver.find_elements(By.XPATH, '//tbody//tr')
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'td')
        data_dict = {}
        for header, cell in zip(header_names_team, cells):
            data_dict[header] = cell.text
        team_stats.append(data_dict)
    
    return team_stats

In [6]:
team_data = scrape_team_stats()

## Mengambil data match

In [7]:
def scrape_match_stats():
    match_stats = []
    PATH = "C:\Program Files (x86)\chromedriver.exe"
    service = Service(PATH)
    driver = webdriver.Chrome(service=service)
    try:
        # Menuju schedule page
        driver.get('https://iblindonesia.com/games/schedule')
        # Tunggu sampai loading sempurna
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'matchDate')))
        # Buat range tanggal bermain IBL regular season yaitu antara 13 Januari 2024 sampai 7 Juli 2024
        start_date = datetime.datetime.strptime("2024-01-13", "%Y-%m-%d")
        end_date = datetime.datetime.strptime("2024-07-07", "%Y-%m-%d")
        delta = datetime.timedelta(days=1)
        # Loop setiap hari pada range tersebut
        current_date = start_date
        while current_date <= end_date:
            date_str = current_date.strftime("%Y-%m-%d")
            driver.execute_script(f"document.getElementById('matchDate').value = '{date_str}';")
            driver.execute_script("document.getElementById('matchDate').dispatchEvent(new Event('change'));")
            time.sleep(2)  
            matches = driver.find_elements(By.CSS_SELECTOR, '.schedule-item') 
            for match in matches:
                try:
                    # Extract waktu
                    date_time_venue = match.find_element(By.CSS_SELECTOR, '.sportsmagazine-fancy-title h2').text
                    date_time_venue_split = date_time_venue.split(' | ')
                    date_time = date_time_venue_split[0].replace('DATE/TIME : ', '')
                    venue = date_time_venue_split[1].replace('VENUE : ', '')
                    # Format tanggal
                    date = date_time.split(' ')[1] + ' ' + date_time.split(' ')[0] + ', ' + date_time.split(' ')[2]
                    # Extract nama tim dan score home
                    home_team = match.find_element(By.CSS_SELECTOR, '.schedule-item-body .col-md-4:nth-child(1) h4 strong').text
                    home_score = match.find_element(By.CSS_SELECTOR, '.schedule-item-body .col-md-4.final h1:nth-child(1) strong').text
                    # Extract nama tim dan score away
                    away_team = match.find_element(By.CSS_SELECTOR, '.schedule-item-body .col-md-4:nth-child(3) h4 strong').text
                    away_score = match.find_element(By.CSS_SELECTOR, '.schedule-item-body .col-md-4.final h1:nth-child(3) strong').text
                    match_data = {
                        "Home": home_team,
                        "Away": away_team,
                        "Home Score": home_score,
                        "Away Score": away_score,
                        "Date": date,
                        "Venue": venue
                    }
                    # Append ke array
                    match_stats.append(match_data)
                except Exception as e:
                    print(f"An error occurred while processing a match: {e}")
            # Berpindah ke hari selanjutnya
            current_date += delta
        return match_stats
    finally:
        driver.quit()

In [8]:
match_stats = scrape_match_stats()

## Mengambil data venue tim IBL 

In [9]:
def get_element_text(element):
    return element.text.strip()

In [10]:
def scrape_venue():
    team_data = []
    # Menuju ke venue page
    PATH = "C:\Program Files (x86)\chromedriver.exe"
    service = Service(PATH)
        
    # Inisialisasi Chrome Driver
    driver = webdriver.Chrome(service=service)
    
    try:
        driver.get("https://en.wikipedia.org/wiki/2024_IBL_Indonesia")
        
        # Mencari table pertama
        table = driver.find_element(By.CSS_SELECTOR, 'table.wikitable')
        rows = table.find_elements(By.CSS_SELECTOR, 'tbody tr')
        
        for row in rows:
            cells = row.find_elements(By.CSS_SELECTOR, 'td')
            if len(cells) >= 3: 
                team = get_element_text(cells[0])
                venue = get_element_text(cells[1])
                city = get_element_text(cells[2])
                
                team_info = {
                    "Team": team,
                    "Venue": venue,
                    "City": city
                }
                team_data.append(team_info)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()
    
    return team_data

In [11]:
venue = scrape_venue()

## Mengambil data info pemain IBL Season 23/24

In [37]:
def scrape_player_info():
    player_data = []
    PATH = "C:/Program Files (x86)/chromedriver.exe"
    service = Service(PATH)
    driver = webdriver.Chrome(service=service)
    try:
        driver.get('https://iblindonesia.com/profile/team')
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.team-name a[href*="profile/team/"]')))
        profile_links = driver.find_elements(By.CSS_SELECTOR, 'div.team-name a[href*="profile/team/"]')
        for index in range(len(profile_links)):
            attempts = 0
            while attempts < 3:
                try:
                    profile_links = driver.find_elements(By.CSS_SELECTOR, 'div.team-name a[href*="profile/team/"]')
                    link = profile_links[index]
                    link_href = link.get_attribute('href')
                    driver.get(link_href)
                    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[href="#playerList"][data-toggle="tab"]')))
                    time.sleep(5) # Delay 5 second untuk waktu loading
                    # Click tombol "Roster"
                    roster_button = driver.find_element(By.CSS_SELECTOR, 'a[href="#playerList"][data-toggle="tab"]')
                    roster_button.click()
                    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'playerList')))    
                    rows = driver.find_elements(By.XPATH, '//div[@id="playerList"]//table//tbody//tr')
                    for row in rows:
                        height = row.find_element(By.XPATH, './th').text.strip()
                        shirt_number = row.find_element(By.XPATH, './td[1]').text.strip()
                        name = row.find_element(By.XPATH, './td[2]//a').text.strip()
                        position = row.find_element(By.XPATH, './td[3]').text.strip()
                        
                        player_info = {
                            "Name": name,
                            "Height": height,
                            "Shirt Number": shirt_number,
                            "Position": position
                        }
                        player_data.append(player_info)
                    break
                except StaleElementReferenceException as e:
                    print(f"StaleElementReferenceException: {e}")
                    attempts += 1
                    time.sleep(1)  
                except TimeoutException as e:
                    print(f"TimeoutException: {e}")
                    break 
                except Exception as e:
                    print(f"Error processing link {link_href}: {e}")
                    break 
            driver.get('https://iblindonesia.com/profile/team')
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.team-name a[href*="profile/team/"]')))
    finally:
        driver.quit()
    return player_data

In [38]:
player_info = scrape_player_info()

Processing link: https://iblindonesia.com/profile/team/126037?season=37811
Extracted 16 players so far.
Processing link: https://iblindonesia.com/profile/team/135953?season=37811
Extracted 31 players so far.
Processing link: https://iblindonesia.com/profile/team/126038?season=37811
Extracted 46 players so far.
Processing link: https://iblindonesia.com/profile/team/143227?season=37811
Extracted 62 players so far.
Processing link: https://iblindonesia.com/profile/team/126042?season=37811
Extracted 79 players so far.
Processing link: https://iblindonesia.com/profile/team/170783?season=37811
Extracted 95 players so far.
Processing link: https://iblindonesia.com/profile/team/126045?season=37811
Extracted 112 players so far.
Processing link: https://iblindonesia.com/profile/team/126036?season=37811
Extracted 130 players so far.
Processing link: https://iblindonesia.com/profile/team/126039?season=37811
Extracted 147 players so far.
Processing link: https://iblindonesia.com/profile/team/170782

## Mengambil data official 

In [34]:
def scrape_management_info():
    management_data = []
    PATH = "C:/Program Files (x86)/chromedriver.exe"
    service = Service(PATH)
    driver = webdriver.Chrome(service=service)

    try:
        driver.get('https://iblindonesia.com/profile/team')
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[href*="profile/team/"]')))
        profile_links = driver.find_elements(By.CSS_SELECTOR, 'a[href*="profile/team/"]')

        for index in range(len(profile_links)):
            attempts = 0
            while attempts < 3:
                try:
                    profile_links = driver.find_elements(By.CSS_SELECTOR, 'a[href*="profile/team/"]')
                    link = profile_links[index]
                    link_href = link.get_attribute('href')
                    driver.get(link_href)
                    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//h2[text()='MANAJEMEN']")))
                    time.sleep(5)  # Allow the content to load

                    management_section = driver.find_element(By.XPATH, "//h2[text()='MANAJEMEN']/following-sibling::*[1]")
                    management_text = management_section.get_attribute('innerHTML')
                    soup = BeautifulSoup(management_text, 'html.parser')
                    lines = soup.find_all('span', style=True)

                    for line in lines:
                        text = line.get_text(strip=True)
                        if ':' in text:
                            name, title = text.split(':', 1)
                            name = name.strip()
                            title = title.strip()
                            management_data.append({name: title})

                    break
                except StaleElementReferenceException as e:
                    print(f"StaleElementReferenceException: {e}")
                    attempts += 1
                    time.sleep(1)
                except TimeoutException as e:
                    print(f"TimeoutException: {e}")
                    break
                except Exception as e:
                    print(f"Error processing link {link_href}: {e}")
                    break
            driver.get('https://iblindonesia.com/profile/team')
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[href*="profile/team/"]')))
    finally:
        driver.quit()

    return management_data

# Export To JSON

In [45]:
# Writing to JSON file
with open("player_stats.json", 'w') as jsonfile:
    json.dump(data, jsonfile, indent=4)

print(f"Data successfully exported to player_stats.json")

Data successfully exported to player_stats.json


In [46]:
# Writing to JSON file
with open("match_stats.json", 'w') as jsonfile:
    json.dump(match_stats, jsonfile, indent=4)

print(f"Data successfully exported to match_stats.json")

Data successfully exported to match_stats.json


In [47]:
# Writing to JSON file
with open("team_standings.json", 'w') as jsonfile:
    json.dump(team_data, jsonfile, indent=4)

print(f"Data successfully exported to team_standings.json")

Data successfully exported to team_standings.json


In [48]:
# Writing to JSON file
with open("venue_detail.json", 'w') as jsonfile:
    json.dump(venue, jsonfile, indent=4)

print(f"Data successfully exported to venue_detail.json")

Data successfully exported to venue_detail.json


In [49]:
# Writing to JSON file
with open("player_info.json", 'w') as jsonfile:
    json.dump(player_info, jsonfile, indent=4)

print(f"Data successfully exported to player_info.json")

Data successfully exported to player_info.json


# Import to Pandas

In [50]:
import pandas as pd

In [51]:
file = 'team_standings.json'
file_venue = 'venue_detail.json'
file_player = 'player_stats.json'
file_player_info = 'player_info.json'
file_match = 'match_stats.json'

In [52]:
team_standings = pd.read_json(file)
venue_pd = pd.read_json(file_venue)
player = pd.read_json(file_player)
player_info = pd.read_json(file_player_info)
match = pd.read_json(file_match)

In [53]:
team_standings

,RANK,TEAM,G,W,L,PTS
0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Dewa United Banten,26.0,22.0,4.0,48.0
2,2.0,Pelita Jaya Jakarta,26.0,21.0,5.0,47.0
3,3.0,Prawira Harum Bandung,26.0,21.0,5.0,47.0
4,4.0,Kesatria Bengawan Solo,26.0,19.0,7.0,45.0
5,5.0,Satria Muda Pertamina Jakarta,26.0,19.0,7.0,45.0
6,6.0,RANS Simba Bogor,26.0,16.0,10.0,42.0
7,7.0,Bali United Basketball,26.0,13.0,13.0,39.0
8,8.0,Borneo Hornbills,26.0,11.0,15.0,37.0
9,9.0,Rajawali Medan,26.0,10.0,16.0,36.0


In [54]:
venue_pd

,Team,Venue,City
0,Amartha Hangtuah,GOR Universitas Negeri Jakarta (UNJ),Jakarta
1,Bali United Basketball,GOR Purna Krida,"Badung, Bali"
2,Bima Perkasa Jogja,Among Rogo Sports Hall,Yogyakarta
3,Borneo Hornbills,GOR Laga Tangkas,"Cibinong, Bogor"
4,Dewa United Banten,Dewa United Arena,Tangerang
5,Kesatria Bengawan Solo,Sritex Arena,Solo
6,Pacific Caesar Surabaya,GOR Pacific Caesar,Surabaya
7,Pelita Jaya Jakarta,Tennis Indoor Senayan,Jakarta
8,Prawira Harum Bandung,C-Tra Prawira Arena,Bandung
9,Rajawali Medan,GOR Universitas Medan,Medan


In [55]:
player

,Player,Game Played,Minutes per game,Field Goal Percentage,3 Points Percentage,2 Points Percentage,Free Throw Percentage,Rebounds per game,Assists per game,Blocks per game,Steals per game,Points per game
0,Stevan Wilfredo Neno,12,257:36,29.6,24.6,45.0,64,1.67,2.75,0.17,0.58,5.83
1,Gunawan Gunawan,14,157:13,21.8,21.4,23.1,67,1.07,0.29,0.00,0.21,2.79
2,Fisyaiful Amir,26,739:03,45.6,27.1,53.1,68,3.73,2.65,0.19,1.46,10.04
3,Kelly Purwanto,20,331:12,28.6,14.3,50.0,63,1.55,4.05,0.15,1.20,1.40
4,Amaluddin Ragol,8,71:31,57.1,60.0,50.0,50,1.38,0.25,0.00,0.50,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...
243,Xavier Allen Alexander,11,328:13,48.8,32.3,54.2,57,11.00,6.73,0.55,1.55,16.91
244,Nemanja Besovic,5,136:28,47.4,50.0,46.8,50,9.80,1.00,1.00,0.40,12.80
245,Amir Darrion Williams,5,98:21,54.8,50.0,55.0,70,9.20,1.20,2.20,0.80,10.80
246,Steven Bernard Lenard Green,15,461:27,39.7,25.7,51.6,69,6.53,6.00,0.07,1.93,18.93


In [56]:
player_info

,Name,Height,Shirt Number,Position
0,Amaluddin Ragol,194,19,C
1,Anthony Denell January Jr.,202,44,C
2,Dandung O'neal Pamungkas,185,6,PG/
3,Diftha Pratama,183,1,SG
4,Fisyaiful Amir,183,17,SG
...,...,...,...,...
223,Nikholas Mahesa R,169,9,PG
224,Randika Aprilian,192,19,PF
225,Steven Bernard Lenard Green,196,10,F/G
226,Teddy Apriyana Romadonsyah,190,11,F


# Data Cleaning

In [61]:
dataframes = {
    'team_standings': team_standings,
    'venue_pd': venue_pd,
    'player': player,
    'player_info': player_info,
    'match': match
}

In [65]:
for name, df in dataframes.items():
    null_count = df.isnull().sum().sum()
    print(f'{name} : Nulls = {null_count}')

team_standings : Nulls = 0
venue_pd : Nulls = 0
player : Nulls = 0
player_info : Nulls = 0
match : Nulls = 0


In [66]:
team_standings_clean = team_standings.dropna()

In [67]:
for name, df in dataframes.items():
    null_count = df.isnull().sum().sum()
    print(f'After cleaning, {name} : Nulls = {null_count}')

After cleaning, team_standings : Nulls = 0
After cleaning, venue_pd : Nulls = 0
After cleaning, player : Nulls = 0
After cleaning, player_info : Nulls = 0
After cleaning, match : Nulls = 0


# Export JSON after Cleaning

In [68]:
# Convert DataFrame to JSON format (dict)
team_standings_file = team_standings_clean.to_dict(orient='records')

# Write to JSON file
with open("team_standings.json", 'w') as jsonfile:
    json.dump(team_standings_file, jsonfile, indent=4)